In [0]:
%run /Workspaces/dars_nic_391419_j3w9t_collab/CCU013_02/atlas/00_Load_helpers

###creates table from joining skinny tables and covid population

In [0]:
%sql
select count(distinct(NHS_NUMBER_DEID)) from dars_nic_391419_j3w9t_collab.ccu013_dp_skinny_patient_23_01_2020

In [0]:
%sql
select count(distinct(person_id_deid)) from dars_nic_391419_j3w9t_collab.ccu013_paper_table_one_56million_denominator

In [0]:
%sql
select count(distinct(person_id_deid)) from dars_nic_391419_j3w9t_collab.ccu013_phe_cohort

2021-11-30 considered as end of follow up, update this value here accordingly if follow up is extended

death_flag: if death date within first year of study 23-01-2020 to 23-01-2021 (for SMR). This flags controls the period of deaths considered for the general population and the study population (each phecode). To change the period, change the date in this flag

In [0]:
skinny_phe_cohort_1ymort = spark.sql("""
SELECT 
  p.NHS_NUMBER_DEID, 
  p.CATEGORISED_ETHNICITY, 
  p.DATE_OF_BIRTH as dob,
  p.SEX as sex,
  p.AGE_AT_COHORT_START as age_23_01_2020,
  c.date_first_event, 
  c.type_first_event,
  t.IMD_quintile,
  CASE
        WHEN CATEGORISED_ETHNICITY = 'White'                              THEN 'White'
        WHEN CATEGORISED_ETHNICITY = 'Asian or Asian British'             THEN 'Asian'
        WHEN CATEGORISED_ETHNICITY = 'Black or Black British'             THEN 'Black'
        WHEN (CATEGORISED_ETHNICITY = 'Mixed'  OR CATEGORISED_ETHNICITY LIKE 'Other%') THEN 'Other'
        ELSE 'Unknown'
      END
      AS ethnicity,
  CASE
        WHEN AGE_AT_COHORT_START >= 0 AND AGE_AT_COHORT_START < 10 THEN '0-9'
        WHEN AGE_AT_COHORT_START >= 10 AND AGE_AT_COHORT_START < 20 THEN '10-19'
        WHEN AGE_AT_COHORT_START >= 20 AND AGE_AT_COHORT_START < 30 THEN '20-29'
        WHEN AGE_AT_COHORT_START >= 30 AND AGE_AT_COHORT_START < 40 THEN '30-39'
        WHEN AGE_AT_COHORT_START >= 40 AND AGE_AT_COHORT_START < 50 THEN '40-49'
        WHEN AGE_AT_COHORT_START >= 50 AND AGE_AT_COHORT_START < 60 THEN '50-59'
        WHEN AGE_AT_COHORT_START >= 60 AND AGE_AT_COHORT_START < 70 THEN '60-69'
        WHEN AGE_AT_COHORT_START >= 70 AND AGE_AT_COHORT_START < 80 THEN '70-79'
        WHEN AGE_AT_COHORT_START >= 80 AND AGE_AT_COHORT_START < 90 THEN '80-89'
        WHEN AGE_AT_COHORT_START >= 90 AND AGE_AT_COHORT_START <= 100 THEN '90+'
      END
      AS age_band_23_01_2020,
  if(p.NHS_NUMBER_DEID = c.person_id_deid, 1, 0) as covid_case,
  if(p.NHS_NUMBER_DEID = c.person_id_deid, c.date_death, p.DATE_OF_DEATH) as death_date,   
  if(p.NHS_NUMBER_DEID = c.person_id_deid, c.date_first_event, '2020-01-23') as start_date,
  if(p.NHS_NUMBER_DEID = c.person_id_deid, c.age, p.AGE_AT_COHORT_START) as age_start_date,
  Least(
  if(p.NHS_NUMBER_DEID = c.person_id_deid, c.date_death, p.DATE_OF_DEATH), 
  if(p.NHS_NUMBER_DEID = c.person_id_deid, DATE_ADD(c.date_first_event, 366), to_date('2021-01-23', 'yyyy-MM-dd')), 
  to_date('2021-11-30', 'yyyy-MM-dd')) as end_date, 
  if(datediff(to_date('2021-01-23', 'yyyy-MM-dd'), if(p.NHS_NUMBER_DEID = c.person_id_deid, c.date_death, p.DATE_OF_DEATH)) >= 0, 1, 0) as death_flag
FROM 
  dars_nic_391419_j3w9t_collab.ccu013_dp_skinny_patient_23_01_2020 p
LEFT JOIN 
  dars_nic_391419_j3w9t_collab.ccu013_phe_cohort c
ON 
  p.NHS_NUMBER_DEID = c.person_id_deid
LEFT JOIN
  dars_nic_391419_j3w9t_collab.ccu013_paper_table_one_56million_denominator t
ON 
  p.NHS_NUMBER_DEID = t.person_id_deid
  """) 
display(skinny_phe_cohort_1ymort)

In [0]:
skinny_phe_cohort_1ymort.createOrReplaceGlobalTempView('ccu013_02_dp_skinny_patient_23_01_2020_phe_cohort_joined_1ymort')

In [0]:
drop_table("ccu013_02_dp_skinny_patient_23_01_2020_phe_cohort_joined_1ymort")

In [0]:
create_table("ccu013_02_dp_skinny_patient_23_01_2020_phe_cohort_joined_1ymort")

In [0]:
%sql
select count(distinct(NHS_NUMBER_DEID)) from dars_nic_391419_j3w9t_collab.ccu013_02_dp_skinny_patient_23_01_2020_phe_cohort_joined_1ymort

In [0]:
%sql 
select * from dars_nic_391419_j3w9t_collab.ccu013_02_dp_skinny_patient_23_01_2020_phe_cohort_joined_1ymort limit 10;

**Add column ethnicity granular (see separate notbeook to populate this information)**

In [0]:
%sql 
select s.*, e.ethnicity_11_group, e.ethnicity_5_group,
CASE
        WHEN e.ethnicity_5_group = 'Asian or Asian British' THEN 'asi_ab'
        WHEN e.ethnicity_5_group = 'Black or Black British' THEN 'bla_bb'
        WHEN e.ethnicity_5_group = 'White' THEN 'whi'
        WHEN e.ethnicity_5_group = 'Mixed' THEN 'mix'
        WHEN e.ethnicity_5_group = 'Other Ethnic Group' THEN 'oth_eth'
        WHEN e.ethnicity_5_group = 'Unknown' THEN 'unkn'
      END
      AS ethn_5,
CASE
        WHEN e.ethnicity_11_group = 'Bangladeshi' THEN 'bangl'
        WHEN e.ethnicity_11_group = 'Black African' THEN 'bla_af'
        WHEN e.ethnicity_11_group = 'Black Caribbean' THEN 'bla_ca'
        WHEN e.ethnicity_11_group = 'Chinese' THEN 'chi'
        WHEN e.ethnicity_11_group = 'Indian' THEN 'ind'
        WHEN e.ethnicity_11_group = 'Pakistani' THEN 'pak'
        WHEN e.ethnicity_11_group = 'Mixed' THEN 'mix'
        WHEN e.ethnicity_11_group = 'Other Asian' THEN 'oth_asi'
        WHEN e.ethnicity_11_group = 'Other Black' THEN 'oth_bl'
        WHEN e.ethnicity_11_group = 'Other Ethnic Group' THEN 'oth_eth'
        WHEN e.ethnicity_11_group = 'Unknown' THEN 'unkn'
        WHEN e.ethnicity_11_group = 'White' THEN 'whi'
      END
      AS ethn_11
from dars_nic_391419_j3w9t_collab.ccu013_02_dp_skinny_patient_23_01_2020_phe_cohort_joined_1ymort s
left join dsa_391419_j3w9t_collab.ccu013_02_ethnicity_granular_ProdDate e
on s.NHS_NUMBER_DEID = e.NHS_NUMBER_DEID 
limit 20; 

In [0]:
skinny_new_eth = spark.sql('''select s.*, e.ethnicity_11_group, e.ethnicity_5_group,
CASE
        WHEN e.ethnicity_5_group = 'Asian or Asian British' THEN 'asi_ab'
        WHEN e.ethnicity_5_group = 'Black or Black British' THEN 'bla_bb'
        WHEN e.ethnicity_5_group = 'White' THEN 'whi'
        WHEN e.ethnicity_5_group = 'Mixed' THEN 'mix'
        WHEN e.ethnicity_5_group = 'Other Ethnic Group' THEN 'oth_eth'
        WHEN e.ethnicity_5_group = 'Unknown' THEN 'unkn'
      END
      AS ethn_5,
CASE
        WHEN e.ethnicity_11_group = 'Bangladeshi' THEN 'bangl'
        WHEN e.ethnicity_11_group = 'Black African' THEN 'bla_af'
        WHEN e.ethnicity_11_group = 'Black Caribbean' THEN 'bla_ca'
        WHEN e.ethnicity_11_group = 'Chinese' THEN 'chi'
        WHEN e.ethnicity_11_group = 'Indian' THEN 'ind'
        WHEN e.ethnicity_11_group = 'Pakistani' THEN 'pak'
        WHEN e.ethnicity_11_group = 'Mixed' THEN 'mix'
        WHEN e.ethnicity_11_group = 'Other Asian' THEN 'oth_asi'
        WHEN e.ethnicity_11_group = 'Other Black' THEN 'oth_bl'
        WHEN e.ethnicity_11_group = 'Other Ethnic Group' THEN 'oth_eth'
        WHEN e.ethnicity_11_group = 'Unknown' THEN 'unkn'
        WHEN e.ethnicity_11_group = 'White' THEN 'whi'
      END
      AS ethn_11
from dars_nic_391419_j3w9t_collab.ccu013_02_dp_skinny_patient_23_01_2020_phe_cohort_joined_1ymort s
left join dsa_391419_j3w9t_collab.ccu013_02_ethnicity_granular_ProdDate e
on s.NHS_NUMBER_DEID = e.NHS_NUMBER_DEID ''')


In [0]:

skinny_new_eth.write.mode("overwrite").option("overwriteSchema", "true").saveAsTable("dsa_391419_j3w9t_collab.ccu013_02_dp_skinny_patient_23_01_2020_phe_cohort_joined_1ymort_eth")

In [0]:
%sql 
select * from dsa_391419_j3w9t_collab.ccu013_02_dp_skinny_patient_23_01_2020_phe_cohort_joined_1ymort_eth limit 3;

In [0]:
%sql 
select count(*) from dsa_391419_j3w9t_collab.ccu013_02_dp_skinny_patient_23_01_2020_phe_cohort_joined_1ymort_eth;

add columns to flag presence in HES APC and whether any diagnosis part of the phecode catalogue: 

* creates hes_apc_record  (1 or 0 if person present or not in HES_APC)

* creates hes_record  (1 or 0 if person as a phecode or not)

* creates hes_apc_phe_record as a combination of the above

    (0: if person is not present in HES_APC; 

    1: if person has admission in HES_APC; 

    2: if person has admission in HES_APC and also in the phecode catalogue) 

HES general:

In [0]:
%sql
CREATE OR REPLACE TEMP VIEW ccu013_apc_person_id_deid as
select distinct(person_id_deid) as person_id_deid 
                            from dars_nic_391419_j3w9t_collab.ccu013_apc_pivot

In [0]:
%sql 
select count(distinct(person_id_deid)) from ccu013_apc_person_id_deid

In [0]:
skinny_hes_apc_flag = spark.sql('''select c.*, 
if(c.NHS_NUMBER_DEID = p.person_id_deid, 1, 0) as hes_apc_record 
from dsa_391419_j3w9t_collab.ccu013_02_dp_skinny_patient_23_01_2020_phe_cohort_joined_1ymort_eth c
left join ccu013_apc_person_id_deid p
on c.NHS_NUMBER_DEID = p.person_id_deid
''')

In [0]:
import pyspark.sql.functions as f

skinny_hes_apc_flag.agg(f.sum('hes_apc_record')).show()

In [0]:
import pyspark.sql.functions as f

skinny_hes_apc_flag.agg(f.count('hes_apc_record')).show()

In [0]:
skinny_hes_apc_flag.write.mode("overwrite").option("overwriteSchema", "true").saveAsTable("dsa_391419_j3w9t_collab.ccu013_02_dp_skinny_patient_23_01_2020_phe_cohort_joined_1ymort_eth")

In [0]:
%sql 
select count(*) from dsa_391419_j3w9t_collab.ccu013_02_dp_skinny_patient_23_01_2020_phe_cohort_joined_1ymort_eth;

In [0]:
%sql 
select * from dsa_391419_j3w9t_collab.ccu013_02_dp_skinny_patient_23_01_2020_phe_cohort_joined_1ymort_eth limit 10;

one row per patient phecode table

In [0]:
%sql
CREATE OR REPLACE TEMP VIEW ccu013_phedatav200_person_id_deid as
select distinct(person_id_deid) as person_id_deid 
                            from dsa_391419_j3w9t_collab.ccu013_phedatav200_unrolled_first_sex_dep

In [0]:
%sql 
select count(distinct(person_id_deid)) from ccu013_phedatav200_person_id_deid

In [0]:
skinny_hes_flag = spark.sql('''select c.*, 
if(c.NHS_NUMBER_DEID = p.person_id_deid, 1, 0) as hes_record 
from dsa_391419_j3w9t_collab.ccu013_02_dp_skinny_patient_23_01_2020_phe_cohort_joined_1ymort_eth c
left join ccu013_phedatav200_person_id_deid p
on c.NHS_NUMBER_DEID = p.person_id_deid
''')

In [0]:
import pyspark.sql.functions as f

skinny_hes_flag.agg(f.sum('hes_record')).show()

In [0]:
import pyspark.sql.functions as f

skinny_hes_flag.agg(f.count('hes_record')).show()

In [0]:
skinny_hes_flag.write.mode("overwrite").option("overwriteSchema", "true").saveAsTable("dsa_391419_j3w9t_collab.ccu013_02_dp_skinny_patient_23_01_2020_phe_cohort_joined_1ymort_eth")

In [0]:
%sql 
select count(*) from dsa_391419_j3w9t_collab.ccu013_02_dp_skinny_patient_23_01_2020_phe_cohort_joined_1ymort_eth;

combined flag: apc and phe

In [0]:
%sql 
SELECT (CASE
        WHEN c.hes_apc_record = 0 AND c.hes_record = 0 THEN 0
        WHEN c.hes_apc_record = 1 AND c.hes_record = 0 THEN 1
        WHEN c.hes_apc_record = 1 AND c.hes_record = 1 THEN 2
        ELSE 9
      END)
      AS hes_apc_phe_record,
      c.*
from dsa_391419_j3w9t_collab.ccu013_02_dp_skinny_patient_23_01_2020_phe_cohort_joined_1ymort_eth c;

In [0]:
skinny_hes_apc_phe_flag = spark.sql('''                                   
SELECT (CASE
        WHEN c.hes_apc_record = 0 AND c.hes_record = 0 THEN 0
        WHEN c.hes_apc_record = 1 AND c.hes_record = 0 THEN 1
        WHEN c.hes_apc_record = 1 AND c.hes_record = 1 THEN 2
        ELSE 9
      END)
      AS hes_apc_phe_record,
      c.*
from dsa_391419_j3w9t_collab.ccu013_02_dp_skinny_patient_23_01_2020_phe_cohort_joined_1ymort_eth c
''')

In [0]:
skinny_hes_apc_phe_flag.write.mode("overwrite").option("overwriteSchema", "true").saveAsTable("dsa_391419_j3w9t_collab.ccu013_02_dp_skinny_patient_23_01_2020_phe_cohort_joined_1ymort_eth")

In [0]:
%sql 
select hes_apc_phe_record, count(distinct(NHS_NUMBER_DEID)) from dsa_391419_j3w9t_collab.ccu013_02_dp_skinny_patient_23_01_2020_phe_cohort_joined_1ymort_eth group by hes_apc_phe_record;